In [1]:
%autosave 60

Autosaving every 60 seconds


# Домашнее задание EDA | МТС Тета
-----
- Нужно провести EDA любого датасета на ваше усмотрение в специально подготовленном ipynb
- В этом ноутбуке будут блоки под разные типы графиков
- Нужно выбрать такой датасет, который позволит построить каждый из этих графиков корректно
- В ноутбуке так же будет дополнительное поле для графика на ваш выбор. В нем должен быть график, который не был построен ранее в ноутбуке (другой тип графика)
- Ноутбук должен отбегать в режиме run all, а так же сохранять все изображения шаблонным образом, как заложено в ноутбуке
- Для каждого графика должны быть написаны выводы
- __Вместе с ноутбуком нужно сдать сохраненные изображения (проверьте корректность их сохранения перед отправкой ДЗ)__
- изображения должны быть в формате png или jpeg. Не сохраняйте изображения в html (так делает plotly)
____
**Критерии оценки**
- Вы построили все основные графики без критических ошибок – 8 баллов
- Вы выполнили пункт 1, а так же построили дополнительный график - 10 баллов
- Доп. баллы выставляются за победу в tinder игре – подробности будут позже. Но чтобы в ней победить, ваш график должен оказаться самым красивым. 
- Доп. Баллы получат топ 10 человек итогового рейтинга. 
---
- Максимум 10 баллов (и +5 дополнительных)
- 1-4 балла: незачет
- 5-6 баллов: 3, незачет
- 7-8 баллов: 4, зачет
- 9-10 баллов: 5, зачет

kde plot

## подготовка

In [2]:
import os
import pandas as pd
import kaggle
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [3]:
# подставьте ваше имя и фамилию в эту переменную как в примере, (например Максим_Шаланкин)
image_name_prefix = 'Татьяна_Попова'

# можно поменять, а можно оставить - сюда будут сохраняться изображения
path_to_save_images = 'im'

In [4]:
os.makedirs(path_to_save_images, exist_ok=True)

In [ ]:
# Скачивание датасета
kaggle.api.dataset_download_files('atharvasoundankar/chocolate-sales', path='chocolate_sales_data', unzip=True)

NameError: name 'kaggle' is not defined

In [6]:
data_path = '/Users/tatianasalamatova/reps/MTS_data_analysis/EDA/chocolate_sales_data/Chocolate Sales.csv'

# Чтение данных в DataFrame
data = pd.read_csv(data_path)

In [7]:
# приведу колонки к нужному формату
data['Amount'] = data['Amount'].replace({'\\$': '', ',': '', ' ': ''}, regex=True).astype(float)
data['Date'] = pd.to_datetime(data['Date'], format='%d-%b-%y')

# сгруппирую данные по стране и дате, суммирую по Amount
df_grouped = data.groupby(['Date', 'Country'])['Amount'].sum().reset_index()

# добавила кумулятивную сумму для каждой страны
df_grouped['Cumamount'] = df_grouped.groupby('Country')['Amount'].cumsum()

# Основные характеристики данных

In [8]:
data.head()

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,2022-01-04,5320.0,180
1,Van Tuxwell,India,85% Dark Bars,2022-08-01,7896.0,94
2,Gigi Bohling,India,Peanut Butter Cubes,2022-07-07,4501.0,91
3,Jan Morforth,Australia,Peanut Butter Cubes,2022-04-27,12726.0,342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,2022-02-24,13685.0,184


In [9]:
data.describe()

,Date,Amount,Boxes Shipped
count,1094,1094.000000,1094.000000
mean,2022-05-03 09:04:56.160877568,5652.308044,161.797989
min,2022-01-03 00:00:00,7.000000,1.000000
25%,2022-03-02 00:00:00,2390.500000,70.000000
50%,2022-05-11 00:00:00,4868.500000,135.000000
75%,2022-07-04 00:00:00,8027.250000,228.750000
max,2022-08-31 00:00:00,22050.000000,709.000000
std,NaN,4102.442014,121.544145


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Sales Person   1094 non-null   object        
 1   Country        1094 non-null   object        
 2   Product        1094 non-null   object        
 3   Date           1094 non-null   datetime64[ns]
 4   Amount         1094 non-null   float64       
 5   Boxes Shipped  1094 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 51.4+ KB


In [11]:
fig1 = px.line(df_grouped, 
               x='Date', 
               y='Cumamount', 
               color='Country', 
               title="Кумулятивная сумма дохода по странам",
               markers=False, 
               line_shape="spline")

fig1.update_traces(line=dict(width=3))

fig1.update_layout(
    template="plotly_dark", 
    xaxis_title="Дата", 
    yaxis_title="Кумулятивная сумма дохода",
    xaxis=dict(tickformat='%d-%b-%y', tickangle=45)
)

fig1.show()

Этот график прикольный с точки зрения его полноты, но показывать такое не очень, потому что тяжело читать линии. То есть он хорош, на мой взгляд, как дашборд (чтобы можно было призумить, где нам удобно). А если статичный, то вот так:

In [12]:
df_filtered = df_grouped[(df_grouped['Date'] >= '2022-01-04') & (df_grouped['Date'] <= '2022-01-25')]

fig1 = px.line(df_filtered, 
               x='Date', 
               y='Cumamount', 
               color='Country', 
               title="Кумулятивная сумма дохода по странам (Диапазон дат)",
               line_shape="spline",
               markers=True, 
               )

fig1.update_traces(line=dict(width=4))


fig1.update_layout(
    template="plotly_dark", 
    xaxis_title="Дата", 
    yaxis_title="Кумулятивная сумма дохода",
    xaxis=dict(
        tickformat='%d-%b-%y', 
        tickangle=45, 
        range=['2022-01-04', '2022-01-20']  # Ограничение оси X
    )
)

fig1.show()

file_path = f"{path_to_save_images}/{image_name_prefix}_image_1.png"

fig1.write_image(file_path, scale=4)

# Вывод - Анализ динамики кумулятивного дохода от продаж сладостей по странам

На представленном графике показано как накапливался доход с продаж сладостей в шести разных странах (Австралия, Канада, Великобритания, США, Индия, Новая Зеландия) в период с 4 января 2022 года по 20 января 2022 года. Это позволяет оценить краткосрочные тенденции и выявить лидеров рынка.

Поскольку это кумулятивная сумма, то видно стабильный возрастающий тренд в течение 16 дней, что говорит о постоянном увеличении дохода. Однако темпы роста различаются.

* В первую половину периода (до 10.01) лишь Великобритания демонстрирует заметный прирост продаж, что указывает на высокую активность потребителей или успешную маркетинговую кампанию. Австралия постепенно тоже набирает обороты, однако темпы ее роста не такие быстрые.

* После 10 января наблюдается резкий скачок кумулятивного дохода во всех странах, что может быть связано с сезонными факторами, праздничными распродажами или другими внешними воздействиями. Наиболее выражен он у Австралии, а наименее - у Индии. Но Австралия в отличие от других стран после 10 января не сбавляет темпы и ее доход растет вплоть до указанной даты - 20 января. В то время как у других стран он прирост то замедляется, то увеличивается.

В итоге Австралия выделяется стабильно высокими темпами роста на протяжении всего периода, что может быть обусловлено эффективной стратегией продаж, высоким спросом на продукцию или другими конкурентными преимуществами.

Также отмечу, что динамика продаж подвержена колебаниям, что указывает на влияние внешних факторов.

* Также отмечу, что у некоторых стран нет данных за те даты, которые есть у других стран. Поэтому я беру данные до 25 января, а потом обрезаю саму ось

## (2) (bar plot или stacked bar plot)

In [13]:
country_sales = data.groupby('Country')['Amount'].sum().reset_index()

country_sales = country_sales.sort_values(by='Amount', ascending=False)

fig2 = px.bar(country_sales, x='Country', y='Amount',
              title="Сумма продаж по странам", color='Country',
              height=600,
              width=1000)

fig2.update_traces(texttemplate='%{y}', textposition='outside')

fig2.update_layout(template="plotly_dark", xaxis_title="Страна", yaxis_title="Сумма продаж")

fig2.show()

file_path = f"{path_to_save_images}/{image_name_prefix}_image_2.png"

fig2.write_image(file_path, scale=4)

# Выводы

Итак, я получила общую сумму продаж за весь период для каждой страны. Как можно было предсказать по line plot - в лидерах Австралия, страна заработала 1,14 млн долларов. Великобритания и Индия делят почетное второе место, заработав около 1.05 млн долларов. Немного, но не сильно от них отстали USA. А вот Канада и Новая Зеландия - расслабились и продали заметно меньше, чем другие страны.

##  (3) kde plot

In [14]:
plasma_6_colors = [
    '#0D0887',
    '#47039F',
    '#9C179E',
    '#D8576B',
    '#FB9F3A',
    '#F0F921'
]
countries = data['Country'].unique()

hist_data = []
group_labels = []

for country in countries:
    hist_data.append(data[data['Country'] == country]['Amount'].values)
    group_labels.append(country)
    
fig3 = ff.create_distplot(hist_data, group_labels, bin_size=1000, colors=plasma_6_colors)

fig3.update_layout(
    title_text='Распределение продаж по странам (KDE Plot)',
    xaxis_title='Сумма продаж',
    yaxis_title='Плотность вероятности',
)

fig3.show()


file_path = f"{path_to_save_images}/{image_name_prefix}_image_3.png"

fig3.write_image(file_path, scale=4)

# Выводы

На графике показано распределение продаж в разных ценовых сегментах для шести стран: Канада, США, Новая Зеландия, Австралия, Индия и Великобритания.

Великобритания имеет наибольшую плотность продаж в ценовых сегментах 1000-2000 и 3000-4000 долларов, что означает, что эти товары наиболее популярны среди британских покупателей.

Индия, в свою очередь, демонстрирует наибольшее количество продаж в ценовом сегменте от 2000 до 3000 долларов.

Австралия имеет наиболее широкий ценовой диапазон товаров, с продажами в сегментах от 2500 до 8000 долларов, при этом пик продаж достаточно размазан. Схожая ситуация наблюдается и в Новой Зеландии, но в ее случае продажи смещены в сторону более низких цен.

США активно продает товары в диапазоне от 2000 до 3000 долларов, но также имеют продажи в сегменте от 4000 до 7000 долларов. Это похоже на ситуацию в Великобритании, но у США продажи в более высоком ценовом диапазоне.

Канада в основном продает товары в нижнем ценовом сегменте.

Примечательно, что все страны в основном сосредоточены на продажах товаров в пределах ценового диапазона от 7 до 70000 долларов. После этого объем продаж резко снижается, что свидетельствует о меньшей популярности товаров более высокой стоимости во всех странах.

##  (4) heatmap plot

In [15]:
# сводная таблица: индексы - продукты, столбцы - страны, значения - количество отправленных коробок
pivot_df = data.pivot_table(values='Boxes Shipped', index='Product', columns='Country', aggfunc='sum', fill_value=0)

# Строим тепловую карту
fig4 = go.Figure(data=go.Heatmap(
    z=pivot_df.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    colorscale="YlGnBu"
))

fig4.update_layout(
    title="Тепловая карта отправленных коробок по продуктам и странам",
    xaxis_title="Страна",
    yaxis_title="Продукт",
    template="plotly_dark"
)

fig4.show()

file_path = f"{path_to_save_images}/{image_name_prefix}_image_4.png"

fig4.write_image(file_path, scale=4)

# Выводы

По heat map видно, что Австралия больше всего продает продукцию "50% Dark Bites", а Канада - "Choco Coated Almonds"

В Индии же более популярен - "Spicy Special Slims" (Spicy - логично:), а в Новой Зеландии - "Mint Chip Choco"

Великобритания больше всего продает коробок "Milk Bars", а США - "Fruit & Nut Bars"

И вообще абсолютным лидером по продажам коробок одного типа шоколода выступает Австралия

##  (5) plot  на ваш выбор

In [16]:
import plotly.express as px

country_sales = data.groupby('Country')['Amount'].sum().reset_index()

pastel_colors = ['#c6e2ff', '#ffb3b3', '#ffcc99', '#d6e1ff', '#f0e68c', '#e2f0cb']

fig = px.pie(country_sales, values='Amount', names='Country', title='Продажи по каждой стране', 
             template='plotly_dark')

fig.update_traces(
    marker=dict(line=dict(color='black', width=2), colors=pastel_colors),
    textinfo='percent+label', 
    pull=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
    textfont=dict(size=20, color='gray')
)

fig.update_layout(
    paper_bgcolor='black',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='white', family='Roboto, sans-serif'),
    title_font=dict(size=24, color='white', family='Roboto, sans-serif'),
    title_x=0.44,
    width=800,
    height=800,
    legend=dict(
        font=dict(size=18, color='white'),
        title=dict(font=dict(size=20))
    )
)

fig.show()

file_path = f"{path_to_save_images}/{image_name_prefix}_image_5.png"

fig.write_image(file_path, scale=4)

# Вывод

Итак, как было показано на Line plot, а также можно было предположить из heat map - Австралия - страна лидер по продажам сладостей.

# Итоговый вывод по всему датасету (по всему EDA)

На основе проведенного анализа данных можно сделать следующие выводы. Австралия занимает лидирующую позицию по продажам шоколадок, в частности, шоколадок бренда N, который представлен в 6 странах: Австралия, Канада, Великобритания, США, Индия и Новая Зеландия. Австралия демонстрирует наибольший рост выручки за период с 2 января 2022 года по 30 августа 2022 года. При этом продажи охватывают широкий диапазон ценовых сегментов, что видно на графике KDE.

Наибольшие проблемы с продажами наблюдаются в Новой Зеландии. Несмотря на то, что она также продает шоколадки в широком ценовом диапазоне (как и Австралия), это распределение смещено в область более низких ценовых сегментов, что ограничивает общие выручки. Кроме того, Новая Зеландия показывает один из самых низких темпов роста продаж, что также характерно для США.

In [17]:
print('well done!')

well done!
